#### Imports

In [369]:
import pandas as pd
import numpy as np

#### Read linkedin csv

In [370]:
linkedin = pd.read_csv('../data/linkedin.csv')
linkedin.sample(3)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
194,3771908283,Head of Data (remote),Verve Group,"· Nueva York, Estados Unidos",275.000 $/año - 425.000 $/año,En remoto,Jornada completa,Director,De 201 a 500 empleados · Servicios de publicidad
269,3775183162,RESEARCH STUDY ASSISTANT,University of Washington,"· Seattle, WA",3.240 $/mes - 3.907 $/mes,Presencial,Jornada completa,Sin experiencia,Más de 10.001 empleados · Enseñanza superior
224,3773350153,Statistical Programmer,Green Key Resources,"· Princeton, NJ",135.200 $/año - 145.600 $/año,Híbrido,Jornada completa,Intermedio,De 201 a 500 empleados · Dotación y selección ...


#### Cleaning 'company_state' column

In [371]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'NC', 'SC', 'CO', 'CT', 'ND', 'SD', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
          'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NJ', 'NY', 'NH', 'NM', 'OH', 'OK', 'OR', 'PA', 'RI', 'TN', 'TX', 'UT',
          'UT', 'VT', 'VA', 'WV', 'WA', 'WI', 'WY']
linkedin['company_state'] = linkedin['company_state'].apply(lambda row: 'Estados Unidos' if row == '· Estados Unidos' else row)
linkedin['company_state'].value_counts()

company_state
Estados Unidos                             33
· Nueva York, NY                           11
· Seattle, WA                               8
· Washington, DC                            7
· Cincinnati, OH                            6
                                           ..
· Newport News, VA                          1
· Morris Plains, NJ                         1
· Washington DC-Baltimore y alrededores     1
· Chesterfield, VA                          1
· Santa Ana, CA                             1
Name: count, Length: 151, dtype: int64

In [372]:
def fclean_company_state(df, states):
    for i in states:
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: i if i in str(row) else row)
    return df

linkedin = fclean_company_state(linkedin, states)

In [373]:
def sclean_company_state(df, states):
    for key, value in states.items():
        linkedin['company_state'] = linkedin['company_state'].apply(lambda row: value if key in str(row) else row)
    return df

left_states = {
    'Virginia': 'VA',
    'Minnesota': 'MN',
    'Washington': 'WA',
    'San Francisco': 'CA',
    'Nebraska': 'NE',
    'California': 'CA',
    'Texas': 'TX',
    'Nueva York': 'NY',
    'Michigan': 'MI',
    'Luisiana': 'LA',
    'Florida': 'FL',
    'Maryland': 'MD',
    'Nevada': 'NV',
    'Oregón': 'OR',
    'Oklahoma': 'OK',
    'Georgia': 'GA',
    'Carolina del Sur': 'SC',
    'Delaware': 'DE',
    'Ohio': 'OH',
    'Arkansas': 'AR',
    'Iowa': 'IA',
    'Carolina del Norte': 'NC',
    'Indiana': 'IN',
    'Nashville': 'TN',
    'Massachusetts': 'MA',
    'Tennessee': 'TN',
    'Enid': 'OK'
}
linkedin = sclean_company_state(linkedin, left_states)

#### Cleaning 'company_state' column

In [374]:
linkedin.sample(5)

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size
239,3773894894,Data Analyst I,Centene Corporation,DE,131.900 $/año - 151.700 $/año,En remoto,Jornada completa,Sin experiencia,Más de 10.001 empleados · Hospitales y atenció...
189,3771503379,Data Analyst,"Business Plan Solutions, LLC",TX,120.000 $/año - 165.000 $/año,Híbrido,Jornada completa,Sin experiencia,De 11 a 50 empleados · Servicios y consultoría...
24,3730766989,Senior Deep Learning Compiler Engineer - MLIR,NVIDIA,Estados Unidos,144.000 $/año - 270.250 $/año,En remoto,Jornada completa,Intermedio,Más de 10.001 empleados · Fabricación de equip...
47,3749346453,Adaptive Optics Senior Software Engineer,TMT International Observatory LLC,CA,78 $/h - 104 $/h,Híbrido,Contrato por obra,Intermedio,De 51 a 200 empleados · Servicios de investiga...
236,3773889315,Data Scientist,Confluent,Estados Unidos,131.900 $/año - 151.700 $/año,En remoto,Jornada completa,Sin experiencia,De 1.001 a 5.000 empleados · Desarrollo de sof...


In [375]:
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto                                                                               117
Híbrido                                                                                 105
Presencial                                                                               69
En remotoCoincide con tus preferencias de empleo. La modalidad laboral es En remoto.      2
Name: count, dtype: int64

Aquí podemos ver que hay valores en la columna que pertenecen a la columna salary_range, por lo tanto tienen que moverse a la columna correspondiente.

In [376]:
linkedin['salary_range'] = linkedin['remote_ratio'].apply(lambda row: str(row) if '$' in str(row) else np.nan)
linkedin['remote_ratio'] = linkedin['remote_ratio'].apply(lambda row: np.nan if '$' in str(row) else str(row))
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto                                                                               117
Híbrido                                                                                 105
Presencial                                                                               69
En remotoCoincide con tus preferencias de empleo. La modalidad laboral es En remoto.      2
nan                                                                                       2
Name: count, dtype: int64

In [377]:
def clean_remote_ratio(df, dict_):
    for key, val in dict_.items():
        df['remote_ratio'] = df['remote_ratio'].apply(lambda row: key if key in str(row) else str(row))
    return df

remote_dict = {
    'Presencial': 0,
    'Híbrido': 50,
    'En remoto': 100
}
linkedin = clean_remote_ratio(linkedin, remote_dict)
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     119
Híbrido       105
Presencial     69
nan             2
Name: count, dtype: int64

The next line should not return any row, that is because the cleaning is working fine.

In [378]:
linkedin[(linkedin['remote_ratio'] == 'Jornada completa') | (linkedin['remote_ratio'] == 'Contrato por obra')]

,job_id,job_title,company_name,company_state,salary_range,remote_ratio,employment_type,experience_level,company_size


In [379]:
linkedin['remote_ratio'].value_counts()

remote_ratio
En remoto     119
Híbrido       105
Presencial     69
nan             2
Name: count, dtype: int64

#### Cleaning 'experience_level' column

In [380]:
linkedin.sample(5)
linkedin['experience_level'].value_counts()

experience_level
Intermedio                 118
Sin experiencia             79
Algo de responsabilidad     49
Prácticas                   20
Director                    11
Ejecutivo                    2
Name: count, dtype: int64

#### Cleaning 'employment_type' column

In [381]:
linkedin['employment_type'].value_counts()

employment_type
Jornada completa                                                                                   183
Contrato por obra                                                                                   54
Media jornada                                                                                       31
Prácticas                                                                                           25
Jornada completaCoincide con tus preferencias de empleo. El tipo de empleo es Jornada completa.      2
Name: count, dtype: int64